In [1]:
from __future__ import print_function
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
## Disable tf future deprecated messages
import logging
logging.getLogger('tensorflow').disabled = True
## Disable tf CUDA messages
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

%matplotlib inline

import sys
import random
import numpy as np
import pandas as pd

import utils
import correction
from models.dense import *
from models.gain import gain
from models.soft_impute import SoftImpute
from models.sinkhorn import OTimputer
from models.mida import mida

import scipy.stats

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer, SimpleImputer

import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest

torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True
dtype = torch.cuda.FloatTensor

## Repeat all experiments for 10 runs

In [2]:
n_runs = 10

## Load data

In [3]:
data_missing, missing_mask, y = utils.load_covid()
## Inject noise
data_missing = utils.inject_noise(data_missing.copy(), noise_rate=.60)
## Replace missing values locations by 0
data_missing = data_missing * missing_mask

Dataset shape: (361, 80)
19.75% missing data
Class distribution: (array([0, 1]), array([195, 166]))


## Run OUR METHOD

In [4]:
params = {
    'nb_batches': 10,
    'reg_noise_std': .03,
    'net_input': 'data_corrupted',
    'net_params': [80, 80],
    'adam_lr': .0001,
    'adam_weight_decay': 0.,
}

ours_accs, ours_aucs = [], []
for i in range(n_runs):
    ours_correction = correction.run(data_missing, 4001, params, y=y, missing_mask=missing_mask, seed=i)
    ## Since our method returns 2 imputations select the best one
    scores_raw = utils.get_scores(ours_correction['raw_out'], y)
    acc = scores_raw['test_balanced_accuracy']
    auc = scores_raw['test_roc_auc_ovo']
    acc = acc.mean()
    acc_std = acc.std()
    auc = auc.mean()
    auc_std = auc.std()
    if 'masked_out' in ours_correction:
        scores_masked = utils.get_scores(ours_correction['masked_out'], y)
        acc_masked = scores_masked['test_balanced_accuracy']
        auc_masked = scores_masked['test_roc_auc_ovo']
        if auc_masked.mean() > auc:
            acc = acc_masked.mean()
            acc_std = acc_masked.std()
            auc = auc_masked.mean()
            auc_std = auc_masked.std()
    print(f'OUR METHOD RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%')
    ours_accs.append(acc.mean())
    ours_aucs.append(auc.mean())
print(f'\nOUR METHOD GLOBAL - acc: {round(np.array(ours_accs).mean() * 100, 2)}% +- {round(np.array(ours_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(ours_aucs).mean() * 100, 2)}% +- {round(np.array(ours_aucs).std() * 100, 2)}%')

Ite 00000 - 0.83 sec - Loss 0.082258 - ACC 66.77% - ACC Mean 66.77% - AUC 70.11% - AUC Mean 70.11% - Deter 000
Ite 00050 - 0.14 sec - Loss 0.079037 - ACC 69.17% - ACC Mean 67.81% - AUC 72.81% - AUC Mean 71.92% - Deter 012
Ite 00100 - 0.14 sec - Loss 0.072901 - ACC 65.07% - ACC Mean 67.26% - AUC 70.17% - AUC Mean 70.43% - Deter 062
Ite 00150 - 0.12 sec - Loss 0.068545 - ACC 66.43% - ACC Mean 66.77% - AUC 70.45% - AUC Mean 70.66% - Deter 112
Ite 00200 - 0.10 sec - Loss 0.067306 - ACC 66.73% - ACC Mean 66.72% - AUC 69.68% - AUC Mean 70.09% - Deter 162
Ite 00250 - 0.11 sec - Loss 0.066672 - ACC 66.97% - ACC Mean 66.56% - AUC 70.32% - AUC Mean 69.81% - Deter 212
Ite 00300 - 0.12 sec - Loss 0.066080 - ACC 66.99% - ACC Mean 66.40% - AUC 70.17% - AUC Mean 69.71% - Deter 262
Ite 00350 - 0.11 sec - Loss 0.065372 - ACC 64.71% - ACC Mean 66.26% - AUC 68.35% - AUC Mean 69.44% - Deter 312
Ite 00400 - 0.11 sec - Loss 0.064543 - ACC 65.65% - ACC Mean 66.16% - AUC 70.72% - AUC Mean 69.23% - Deter 362
I

## Run Median, Mean and KNN

In [5]:
data_missing_nans = np.where(missing_mask, data_missing, np.nan)

median_imputer = SimpleImputer(missing_values=np.nan, strategy='median')
mean_imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
knn_imputer = KNNImputer(n_neighbors=10, weights='uniform')

median_completed = median_imputer.fit_transform(data_missing_nans)
mean_completed = mean_imputer.fit_transform(data_missing_nans)
knn_completed = knn_imputer.fit_transform(data_missing_nans)

## All runs would be the same since deterministic methods
scores = utils.get_scores(median_completed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MEDIAN - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
median_accs = acc
median_aucs = auc

scores = utils.get_scores(mean_completed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MEAN - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
mean_accs = acc
mean_aucs = auc

scores = utils.get_scores(knn_completed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'KNN - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
knn_accs = acc
knn_aucs = auc

MEDIAN - acc: 67.16% +- 0.0% - auc: 72.08% +- 0.0%
MEAN - acc: 65.52% +- 0.0% - auc: 71.04% +- 0.0%
KNN - acc: 65.44% +- 0.0% - auc: 71.62% +- 0.0%


## Run GAIN

In [6]:
gain_parameters = {'batch_size': 128,
                   'hint_rate': .9,
                   'alpha': 100,
                   'iterations': 5000}

gain_accs, gain_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputed = gain(data_missing_nans, gain_parameters)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nGAIN RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    gain_accs.append(acc.mean())
    gain_aucs.append(auc.mean())
print(f'GAIN GLOBAL - acc: {round(np.array(gain_accs).mean() * 100, 2)}% +- {round(np.array(gain_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(gain_aucs).mean() * 100, 2)}% +- {round(np.array(gain_aucs).std() * 100, 2)}%')

100%|██████████| 5000/5000 [00:36<00:00, 136.98it/s]



GAIN RUN 1/10 - acc: 64.55% +- 4.06% - auc: 72.23% +- 4.14%



100%|██████████| 5000/5000 [00:36<00:00, 138.05it/s]



GAIN RUN 2/10 - acc: 65.8% +- 5.7% - auc: 70.93% +- 6.28%



100%|██████████| 5000/5000 [00:36<00:00, 136.98it/s]


...


GAIN RUN 8/10 - acc: 68.38% +- 4.86% - auc: 74.54% +- 7.53%



100%|██████████| 5000/5000 [00:36<00:00, 138.72it/s]



GAIN RUN 9/10 - acc: 68.31% +- 3.69% - auc: 72.8% +- 5.53%



100%|██████████| 5000/5000 [00:40<00:00, 122.56it/s]



GAIN RUN 10/10 - acc: 70.64% +- 4.94% - auc: 76.43% +- 4.91%

GAIN GLOBAL - acc: 67.32% +- 2.36% - auc: 73.36% +- 1.93%


## Run MICE

In [7]:
imputer = IterativeImputer()
imputed = imputer.fit_transform(data_missing_nans)
## All runs would be the same since deterministic method
scores = utils.get_scores(imputed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MICE - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
mice_accs = acc
mice_aucs = auc

MICE - acc: 67.95% +- 0.0% - auc: 71.94% +- 0.0%


## Run MISSFOREST

In [8]:
miss_accs, miss_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputer = MissForest(n_jobs=-1, random_state=i)
    imputed = imputer.fit_transform(data_missing_nans)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nMISSFOREST RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    miss_accs.append(acc.mean())
    miss_aucs.append(auc.mean())
print(f'\nMISSFOREST GLOBAL - acc: {round(np.array(miss_accs).mean() * 100, 2)}% +- {round(np.array(miss_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(miss_aucs).mean() * 100, 2)}% +- {round(np.array(miss_aucs).std() * 100, 2)}%')

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6

MISSFOREST RUN 1/10 - acc: 66.26% +- 5.48% - auc: 71.98% +- 8.21%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6

MISSFOREST RUN 2/10 - acc: 64.97% +- 3.24% - auc: 71.55% +- 3.29%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5

MISSFOREST RUN 3/10 - acc: 68.48% +- 5.95% - auc: 73.7% +- 5.69%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6

MISSFOREST RUN 4/10 - acc: 68.27% +- 4.11% - auc: 74.11% +- 4.54%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8

MISSFOREST RUN 5/10 - acc: 66.28% +- 3.13% - auc: 72.09% +- 3.69%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5

MISSFOREST RUN 6/10 - acc: 66.71% +- 4.62% - auc: 73.2% +- 5.53%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration

## Run SOFTIMPUTE

In [9]:
soft_accs, soft_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputer = SoftImpute(random_state=i)
    imputer.fit(data_missing_nans)
    imputed = imputer.predict(data_missing_nans)
    imputed = np.where(missing_mask, data_missing, imputed)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'SOFTIMPUTE RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%')
    soft_accs.append(acc.mean())
    soft_aucs.append(auc.mean())
print(f'\nSOFTIMPUTE GLOBAL - acc: {round(np.array(soft_accs).mean() * 100, 2)}% +- {round(np.array(soft_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(soft_aucs).mean() * 100, 2)}% +- {round(np.array(soft_aucs).std() * 100, 2)}%')

SOFTIMPUTE RUN 1/10 - acc: 64.8% +- 5.0% - auc: 72.88% +- 7.76%
SOFTIMPUTE RUN 2/10 - acc: 67.47% +- 6.24% - auc: 75.26% +- 6.75%
SOFTIMPUTE RUN 3/10 - acc: 68.16% +- 3.57% - auc: 72.09% +- 2.99%
SOFTIMPUTE RUN 4/10 - acc: 66.97% +- 3.28% - auc: 74.82% +- 5.31%
SOFTIMPUTE RUN 5/10 - acc: 64.08% +- 2.92% - auc: 72.9% +- 2.66%
SOFTIMPUTE RUN 6/10 - acc: 66.62% +- 5.38% - auc: 73.7% +- 5.71%
SOFTIMPUTE RUN 7/10 - acc: 66.34% +- 4.02% - auc: 71.89% +- 6.89%
SOFTIMPUTE RUN 8/10 - acc: 66.28% +- 5.31% - auc: 71.89% +- 5.69%
SOFTIMPUTE RUN 9/10 - acc: 66.03% +- 4.11% - auc: 74.39% +- 6.72%
SOFTIMPUTE RUN 10/10 - acc: 65.17% +- 2.23% - auc: 73.26% +- 5.06%

SOFTIMPUTE GLOBAL - acc: 66.19% +- 1.18% - auc: 73.31% +- 1.15%


## Run SINKHORN

In [10]:
sink_accs, sink_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputer = OTimputer(niter=500)
    imputed = imputer.fit_transform(data_missing_nans).cpu().detach().numpy()
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nSINKHORN RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    sink_accs.append(acc.mean())
    sink_aucs.append(auc.mean())
print(f'\nSINKHORN GLOBAL - acc: {round(np.array(sink_accs).mean() * 100, 2)}% +- {round(np.array(sink_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(sink_aucs).mean() * 100, 2)}% +- {round(np.array(sink_aucs).std() * 100, 2)}%')

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 2.7988

SINKHORN RUN 1/10 - acc: 67.66% +- 4.71% - auc: 74.61% +- 6.14%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 2.6428

SINKHORN RUN 2/10 - acc: 69.69% +- 4.24% - auc: 73.21% +- 4.17%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 2.7103

SINKHORN RUN 3/10 - acc: 65.54% +- 3.32% - auc: 72.5% +- 5.42%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 2.4630

SINKHORN RUN 4/10 - acc: 68.31% +- 5.1% - auc: 75.22% +- 5.91%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 2.8821

SINKHORN RUN 5/10 - acc: 65.95% +- 7.51% - auc: 72.01% +- 6.26%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 2.6999

SINKHORN RUN 6/10 - acc: 66.95% +- 1.86% - auc: 73.17% +- 3.19%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 2.5172

SINKHORN RUN 7/10 - acc: 68.95% +- 5.05% - auc: 76.49% +- 5.24%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 2.5824

SINKHORN RUN 8/10 - acc: 64.82% +- 6.24% - auc: 70.

## Run MIDA

In [11]:
mida_accs, mida_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputed = mida(data_missing_nans, num_layers=2, num_epochs=1500)
    imputed = np.where(missing_mask, data_missing, imputed)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nMIDA RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    mida_accs.append(acc.mean())
    mida_aucs.append(auc.mean())
print(f'\nMIDA GLOBAL - acc: {round(np.array(mida_accs).mean() * 100, 2)}% +- {round(np.array(mida_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(mida_aucs).mean() * 100, 2)}% +- {round(np.array(mida_aucs).std() * 100, 2)}%')

Stop training at epoch: 1500/1500, return best output

MIDA RUN 1/10 - acc: 65.94% +- 5.3% - auc: 71.42% +- 4.51%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 2/10 - acc: 67.04% +- 5.73% - auc: 72.2% +- 7.06%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 3/10 - acc: 65.84% +- 5.48% - auc: 72.03% +- 6.87%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 4/10 - acc: 66.31% +- 2.75% - auc: 69.82% +- 3.96%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 5/10 - acc: 66.73% +- 4.91% - auc: 70.16% +- 7.01%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 6/10 - acc: 68.25% +- 4.39% - auc: 72.69% +- 4.53%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 7/10 - acc: 66.98% +- 4.23% - auc: 74.5% +- 5.01%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 8/10 - acc: 66.52% +- 4.43% - auc: 72.41% +- 5.1%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 9/10 - acc: 

## Run T-tests

In [12]:
for model, metrics in {
        'MEDIAN': {'ACC': median_accs, 'AUC': median_aucs},
        'MEAN': {'ACC': mean_accs, 'AUC': mean_aucs},
        'KNN': {'ACC': knn_accs, 'AUC': knn_aucs},
        'GAIN': {'ACC': gain_accs, 'AUC': gain_aucs},
        'MICE': {'ACC': mice_accs, 'AUC': mice_aucs},
        'MISS': {'ACC': miss_accs, 'AUC': miss_aucs},
        'SOFT': {'ACC': soft_accs, 'AUC': soft_aucs},
        'SINK': {'ACC': sink_accs, 'AUC': sink_aucs},
        'MIDA': {'ACC': mida_accs, 'AUC': mida_aucs}}.items():
    for metric_name, metric in metrics.items():
        ours_metric = ours_accs if metric_name == 'ACC' else ours_aucs
        t, p = scipy.stats.ttest_ind(np.array(ours_metric), np.array(metric))
        if p <= .05:
            if t > 0:
                ## Our method is better
                print(f'Metric {metric_name} - OUR METHOD is significantly better than {model}')
            else:
                ## Theirs is better
                print(f'Metric {metric_name} - OUR METHOD is significantly worse than {model}')
        else:
            ## Else we are even
            print(f'Metric {metric_name} - OUR METHOD is even with {model}')

Metric ACC - OUR METHOD is significantly better than MEDIAN
Metric AUC - OUR METHOD is significantly better than MEDIAN
Metric ACC - OUR METHOD is significantly better than MEAN
Metric AUC - OUR METHOD is significantly better than MEAN
Metric ACC - OUR METHOD is significantly better than KNN
Metric AUC - OUR METHOD is significantly better than KNN
Metric ACC - OUR METHOD is even with GAIN
Metric AUC - OUR METHOD is even with GAIN
Metric ACC - OUR METHOD is significantly better than MICE
Metric AUC - OUR METHOD is significantly better than MICE
Metric ACC - OUR METHOD is significantly better than MISS
Metric AUC - OUR METHOD is significantly better than MISS
Metric ACC - OUR METHOD is significantly better than SOFT
Metric AUC - OUR METHOD is even with SOFT
Metric ACC - OUR METHOD is significantly better than SINK
Metric AUC - OUR METHOD is even with SINK
Metric ACC - OUR METHOD is significantly better than MIDA
Metric AUC - OUR METHOD is significantly better than MIDA
